In [32]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [33]:
csv_file = "eye_diagrams_metadata.csv"
df = pd.read_csv(csv_file)

# Define paths
train_folder = "preprocessed/train"
test_folder = "preprocessed/test"

# Change target variable to OSNR_dB
target_variable = "OSNR_dB"


In [34]:
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["OSNR_dB"], random_state=42)

# Image size and batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [35]:
def load_images_and_labels(folder, df_subset):
    images = []
    labels = []
    
    for _, row in tqdm(df_subset.iterrows(), total=len(df_subset), desc=f"Loading images from {folder}"):
        filename = row["Filename"].replace(".png", ".npy")  # Adjust filename for .npy
        file_path = os.path.join(folder, filename)
        
        if not os.path.exists(file_path):
            print(f"⚠️ Skipping missing file: {filename}")
            continue
        
        # Load image
        image = np.load(file_path)
        image = np.expand_dims(image, axis=-1)  # Add channel dimension for CNN

        images.append(image)
        labels.append(row[target_variable])  # OSNR_dB values
    
    return np.array(images), np.array(labels)

# Load dataset
X_train, y_train = load_images_and_labels(train_folder, train_df)
X_test, y_test = load_images_and_labels(test_folder, test_df)


Loading images from preprocessed/test: 100%|██████████| 123/123 [00:00<00:00, 1594.05it/s]


In [36]:
X_train = X_train / 255.0
X_test = X_test / 255.0

# Standardize labels (OSNR values)
scaler = StandardScaler()
y_train = scaler.fit_transform(y_train.reshape(-1, 1))
y_test = scaler.transform(y_test.reshape(-1, 1))

In [37]:
datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1)
datagen.fit(X_train)

In [38]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber

model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(512, (3, 3), activation='relu'),  # Added another Conv layer
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dense(1, activation='linear')  # Linear activation for regression
])

# Compile the model using Huber Loss (better for regression)
model.compile(optimizer=Adam(learning_rate=0.0001), loss=Huber(), metrics=['mae'])

In [ ]:
print("\n🚀 Training Improved CNN Model for OSNR Prediction...")
history = model.fit(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE), 
                    epochs=10, 
                    validation_data=(X_test, y_test))

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)


🚀 Training Improved CNN Model for OSNR Prediction...
Epoch 1/10
16/16 [==============================] - 89s 5s/step - loss: 0.4505 - mae: 0.8253 - val_loss: 0.4525 - val_mae: 0.8255
Epoch 2/10
16/16 [==============================] - 89s 6s/step - loss: 0.4494 - mae: 0.8282 - val_loss: 0.4522 - val_mae: 0.8321
Epoch 3/10
16/16 [==============================] - 92s 6s/step - loss: 0.4489 - mae: 0.8285 - val_loss: 0.4522 - val_mae: 0.8326
Epoch 4/10
16/16 [==============================] - 87s 5s/step - loss: 0.4530 - mae: 0.8369 - val_loss: 0.4524 - val_mae: 0.8362
Epoch 5/10
16/16 [==============================] - 95s 6s/step - loss: 0.4491 - mae: 0.8321 - val_loss: 0.4522 - val_mae: 0.8327
Epoch 6/10
16/16 [==============================] - 106s 7s/step - loss: 0.4491 - mae: 0.8297 - val_loss: 0.4524 - val_mae: 0.8357
Epoch 7/10
 8/16 [==============>...............] - ETA: 49s - loss: 0.4724 - mae: 0.8562

In [ ]:
# Convert back from standardized values
y_test_actual = scaler.inverse_transform(y_test)
y_pred_actual = scaler.inverse_transform(y_pred)

# Calculate additional accuracy metrics
mae_actual = mean_absolute_error(y_test_actual, y_pred_actual)
mse_actual = mean_squared_error(y_test_actual, y_pred_actual)
r2 = r2_score(y_test_actual, y_pred_actual)

# Print accuracy metrics
print("\n✅ CNN OSNR Prediction Model Evaluation:")
print(f"📏 Mean Absolute Error (MAE): {mae_actual:.4f}")
print(f"📏 Mean Squared Error (MSE): {mse_actual:.4f}")
print(f"📏 R² Score: {r2:.4f} (Closer to 1 is better)")

In [ ]:
# Save the trained model
model.save("cnn_osnr_model.h5")
print("✅ CNN model saved as 'cnn_osnr_model.h5'")

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test_actual, y_pred_actual, alpha=0.5, color='blue')
plt.plot([min(y_test_actual), max(y_test_actual)], [min(y_test_actual), max(y_test_actual)], linestyle='--', color='red')
plt.xlabel("Actual OSNR (dB)")
plt.ylabel("Predicted OSNR (dB)")
plt.title("Actual vs Predicted OSNR Values")
plt.show()